**Семантическая маршрутизация**

**1. Создайте функцию для подключения энкодера**

Напишите функцию init_sentence_encoder, которая подключит предварительно обученную модель SentenceTransformer из каталога моделей Hugging Face. Мы будем использовать уменьшенную версию BERT для русского языка — cointegrated/rubert-tiny2.



**2. Создайте класс для маршрутов**

Теперь создайте класс Route, который будет описывать определённое намерение пользователя и содержать список фраз, связанных с этим намерением.

При создании маршрута нужно будет рассчитать эмбеддинги для всех фраз, используя энкодер предложений, который вы сделали на первом шаге. Для удобства сохраните эти эмбеддинги как torch.Tensor (см. документацию метода encode).



## **3. Создайте класс для семантического маршрутизатора**

Теперь подготовим класс SemanticRouter. Этот класс будет содержать все ваши маршруты и помогать определять намерения пользователя по его запросу.

Запрос пользователя будет переводиться в эмбеддинг с помощью модели энкодера предложений. Затем мы будем сравнивать этот эмбеддинг с эмбеддингами всех маршрутов, чтобы найти тот, который больше всего похож на запрос.

Для сравнения используйте косинусное сходство — есть специальный метод для его расчёта между двумя тензорами (torch.Tensor) в модуле util.

Если запрос пользователя не достаточно похож на фразы в маршрутах, верните None - мы не можем уверенно сопоставить запрос с каким-либо из известных намерений.

In [8]:
from typing import List, Optional
from sentence_transformers import SentenceTransformer, util
import torch


def init_sentence_encoder(model_name: str = 'cointegrated/rubert-tiny2') -> SentenceTransformer:
    """
    Initialize a SentenceTransformer model for encoding sentences.

    Parameters:
        model_name (str): The name of the model to load. Default is 'cointegrated/rubert-tiny2'.
            Model names can be found at the Hugging Face model hub: https://huggingface.co/models

    Returns:
        SentenceTransformer: An initialized SentenceTransformer model.

    Raises:
        ValueError: If the model name is empty.
        RuntimeError: If the model fails to load.
    """
    if not model_name:
        raise ValueError("Model name must not be empty")
    try:
        model = SentenceTransformer(model_name)
        return model
    except Exception as e:
        raise RuntimeError(f"Failed to initialize model '{model_name}': {e}")


class Route:
    """
    A class representing a route, which consists of a name and a list of sentences.

    Attributes:
        name (str): The name of the route.
        sentences (List[str]): A list of sentences representing the route.
        embeddings (torch.Tensor): Embeddings of the sentences generated by the SentenceTransformer.
    """

    def __init__(self, name: str, sentences: List[str]):
        """
        Initialize a Route instance.

        Parameters:
            name (str): The name of the route.
            sentences (List[str]): A list of sentences for the route.

        Raises:
            ValueError: If the route name is empty or the sentences list is empty or contains empty sentences.
            RuntimeError: If there is an error encoding the sentences.
        """
        if not name:
            raise ValueError("Route name must not be empty")
        if not sentences or not all(isinstance(s, str) and s.strip() for s in sentences):
            raise ValueError("Sentences list must not be empty and should not contain empty sentences")

        self.name = name
        self.sentences = sentences

        # Инициализируем энкодер
        self.encoder = init_sentence_encoder()

        try:
            self.embeddings = self.encoder.encode(sentences, convert_to_tensor=True)
        except Exception as e:
            raise RuntimeError(f"Failed to encode sentences for route '{name}': {e}")


class SemanticRouter:
    """
    A class representing a semantic router for classifying user intents.
    """

    def __init__(self, routes: List[Route]):
        """
        Initialize a SemanticRouter instance.

        Parameters:
            routes (List[Route]): A list of Route objects.
        """
        if not routes:
            raise ValueError("Routes list must not be empty")
        self.routes = routes
        self.encoder = routes[0].encoder  # Используем энкодер из первого маршрута

    def classify_intent(self, user_input: str, similarity_threshold: float = 0.8) -> Optional[str]:
        """
        Classify the intent of a user input by comparing it to predefined routes.

        Parameters:
            user_input (str): The user's input to classify.
            similarity_threshold (float): The threshold for similarity to consider a match. Default is 0.8.

        Returns:
            Optional[str]: The name of the best matching route if the similarity exceeds the threshold, otherwise None.
        """
        if not user_input:
            return None

        try:
            input_embedding = self.encoder.encode(user_input, convert_to_tensor=True)
        except Exception as e:
            raise RuntimeError(f"Failed to encode user input: {e}")

        best_match = None
        best_score = 0.0

        for route in self.routes:
            similarities = util.cos_sim(input_embedding, route.embeddings)
            max_score = torch.max(similarities).item()
            if max_score > best_score and max_score >= similarity_threshold:
                best_score = max_score
                best_match = route.name

        return best_match

In [9]:
sentence_encoder = init_sentence_encoder()

b2c_support = Route(
    name="b2c_support",
    sentences=[
        "Какие способы оплаты вы принимаете?",
        "Как использовать промокод при оплате?",
        "Можно ли оплатить курс в рассрочку?",
        "Почему мне дважды списали деньги за курс?"
    ],
)

tech_support = Route(
    name="tech_support",
    sentences=[
        "Что делать, если не приходит подтверждение на email?",
        "Как устранить проблемы с воспроизведением видео?",
        "Почему не отображаются прогресс и оценки?",
        "Почему мой аккаунт не активирован?"
    ],
)

router = SemanticRouter([b2c_support, tech_support])

print(router.classify_intent('Как изменить способ оплаты?'))

b2c_support
